Evaluate the performance of different types of optimizer on a LeNet-5 network using MNIST data. At least you need to evaluate SGD, AdaGrad, RMSprop. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

In [2]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Preparing for Data
print('==> Preparing data..')

"""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""

==> Preparing data..


"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n"

In [4]:
# Preparing for Data
print('==> Preparing data..')

# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])


==> Preparing data..


In [5]:
#Defining the convolutional neural network
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = torch.flatten(out, 1) # flatten all dimensions except batch
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
 
        return out

In [6]:
model1 = LeNet()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 61,706 trainable parameters


In [8]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

#import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    count = 0
    train_loss = 0
    total = 0 
    correct = 0
    
    total_step = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        

        #forward pass
        y_pred = model(data)

        loss = criterion(y_pred, target)

        acc = calculate_accuracy(y_pred, target)
        
        
        #Backward pass
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [10]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [12]:
#checking the dataset in the code 
trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [13]:
#printing the dataset and dimensions to check whether we are working on the correct dataset:

for data, target in train_loader:
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', target.shape)
    print('Class labels of the 10 examples:', target[:10])
    break
    

Image batch dimensions: torch.Size([128, 1, 32, 32])
Image label dimensions: torch.Size([128])
Class labels of the 10 examples: tensor([7, 6, 7, 2, 2, 1, 7, 0, 6, 3])


In [14]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"mnist_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298496
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300692
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.279521
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.226394
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.463630
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.290723
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.601500
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.431361
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.154827
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.214622
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.160757
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.243699
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.121756
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.194640
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.221097
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.133170
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.147094
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.061605
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.047299
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.209820
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.036913
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.009663
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.036477
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.057542
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.073799
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.021350
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.013170
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.036468
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.032905
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.080626
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.070895
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.048957
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.040905
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.019482
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.021651
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.037171
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.011517
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.026594
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.004974
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.025808


Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.014207
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.081361
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.028540
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.057037
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.009111

Test set: Average loss: -14.3072, Accuracy: 9912/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.065144
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.003629
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.037491
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.002568
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.009048
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.024695
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.006547
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.004115
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.018373
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.016344
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.019844
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.029290
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.003579
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.013745
Tra

Train Epoch: 11 [16640/60000 (28%)]	Loss: 0.029774
Train Epoch: 11 [17920/60000 (30%)]	Loss: 0.025821
Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.056332
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.017358
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.012852
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.014886
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.021641
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.007913
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.007383
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.004991
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.011014
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.005899
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.026279
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.016534
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.042578
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.029527
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.009349
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.006685
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.005795
Train Epoch: 11 [40960/60000 (6

Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.041148
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.006976
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.002620
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.035541
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.001564
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.026704
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.013348
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.014588
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.016310
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.007392
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.029958
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.009455
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.009513
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.017351
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.002067
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.035048
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.006470

Test set: Average loss: -14.1384, Accuracy: 9896/10000 (99%)

Train Epoch: 15 [0/60000 (0%)]	Loss: 0.013385
Train Epoch: 15 [1280/60

...

Train Epoch: 34 [47360/60000 (79%)]	Loss: 0.044290
Train Epoch: 34 [48640/60000 (81%)]	Loss: 0.003667
Train Epoch: 34 [49920/60000 (83%)]	Loss: 0.010966
Train Epoch: 34 [51200/60000 (85%)]	Loss: 0.010594
Train Epoch: 34 [52480/60000 (87%)]	Loss: 0.023181
Train Epoch: 34 [53760/60000 (90%)]	Loss: 0.091297
Train Epoch: 34 [55040/60000 (92%)]	Loss: 0.020167
Train Epoch: 34 [56320/60000 (94%)]	Loss: 0.035534
Train Epoch: 34 [57600/60000 (96%)]	Loss: 0.001590
Train Epoch: 34 [58880/60000 (98%)]	Loss: 0.014961

Test set: Average loss: -13.8313, Accuracy: 9907/10000 (99%)

Train Epoch: 35 [0/60000 (0%)]	Loss: 0.026276
Train Epoch: 35 [1280/60000 (2%)]	Loss: 0.011934
Train Epoch: 35 [2560/60000 (4%)]	Loss: 0.009867
Train Epoch: 35 [3840/60000 (6%)]	Loss: 0.038218
Train Epoch: 35 [5120/60000 (9%)]	Loss: 0.001855
Train Epoch: 35 [6400/60000 (11%)]	Loss: 0.005472
Train Epoch: 35 [7680/60000 (13%)]	Loss: 0.001568
Train Epoch: 35 [8960/60000 (15%)]	Loss: 0.011083
Train Epoch: 35 [10240/60000 (17%)]

Train Epoch: 38 [7680/60000 (13%)]	Loss: 0.003959
Train Epoch: 38 [8960/60000 (15%)]	Loss: 0.013171
Train Epoch: 38 [10240/60000 (17%)]	Loss: 0.008664
Train Epoch: 38 [11520/60000 (19%)]	Loss: 0.002355
Train Epoch: 38 [12800/60000 (21%)]	Loss: 0.012154
Train Epoch: 38 [14080/60000 (23%)]	Loss: 0.005020
Train Epoch: 38 [15360/60000 (26%)]	Loss: 0.005959
Train Epoch: 38 [16640/60000 (28%)]	Loss: 0.003693
Train Epoch: 38 [17920/60000 (30%)]	Loss: 0.039980
Train Epoch: 38 [19200/60000 (32%)]	Loss: 0.010130
Train Epoch: 38 [20480/60000 (34%)]	Loss: 0.000578
Train Epoch: 38 [21760/60000 (36%)]	Loss: 0.019647
Train Epoch: 38 [23040/60000 (38%)]	Loss: 0.010187
Train Epoch: 38 [24320/60000 (41%)]	Loss: 0.003623
Train Epoch: 38 [25600/60000 (43%)]	Loss: 0.000298
Train Epoch: 38 [26880/60000 (45%)]	Loss: 0.002396
Train Epoch: 38 [28160/60000 (47%)]	Loss: 0.008064
Train Epoch: 38 [29440/60000 (49%)]	Loss: 0.003009
Train Epoch: 38 [30720/60000 (51%)]	Loss: 0.006578
Train Epoch: 38 [32000/60000 (53%

Train Epoch: 41 [29440/60000 (49%)]	Loss: 0.012548
Train Epoch: 41 [30720/60000 (51%)]	Loss: 0.005189
Train Epoch: 41 [32000/60000 (53%)]	Loss: 0.006484
Train Epoch: 41 [33280/60000 (55%)]	Loss: 0.002666
Train Epoch: 41 [34560/60000 (58%)]	Loss: 0.000851
Train Epoch: 41 [35840/60000 (60%)]	Loss: 0.004355
Train Epoch: 41 [37120/60000 (62%)]	Loss: 0.012247
Train Epoch: 41 [38400/60000 (64%)]	Loss: 0.002059
Train Epoch: 41 [39680/60000 (66%)]	Loss: 0.009095
Train Epoch: 41 [40960/60000 (68%)]	Loss: 0.006102
Train Epoch: 41 [42240/60000 (70%)]	Loss: 0.023629
Train Epoch: 41 [43520/60000 (72%)]	Loss: 0.001215
Train Epoch: 41 [44800/60000 (75%)]	Loss: 0.004366
Train Epoch: 41 [46080/60000 (77%)]	Loss: 0.005956
Train Epoch: 41 [47360/60000 (79%)]	Loss: 0.008640
Train Epoch: 41 [48640/60000 (81%)]	Loss: 0.004868
Train Epoch: 41 [49920/60000 (83%)]	Loss: 0.003707
Train Epoch: 41 [51200/60000 (85%)]	Loss: 0.055879
Train Epoch: 41 [52480/60000 (87%)]	Loss: 0.013548
Train Epoch: 41 [53760/60000 (9

Train Epoch: 44 [51200/60000 (85%)]	Loss: 0.015303
Train Epoch: 44 [52480/60000 (87%)]	Loss: 0.028473
Train Epoch: 44 [53760/60000 (90%)]	Loss: 0.021550
Train Epoch: 44 [55040/60000 (92%)]	Loss: 0.004362
Train Epoch: 44 [56320/60000 (94%)]	Loss: 0.017552
Train Epoch: 44 [57600/60000 (96%)]	Loss: 0.006540
Train Epoch: 44 [58880/60000 (98%)]	Loss: 0.115305

Test set: Average loss: -13.7782, Accuracy: 9904/10000 (99%)

Train Epoch: 45 [0/60000 (0%)]	Loss: 0.014633
Train Epoch: 45 [1280/60000 (2%)]	Loss: 0.003519
Train Epoch: 45 [2560/60000 (4%)]	Loss: 0.035248
Train Epoch: 45 [3840/60000 (6%)]	Loss: 0.015991
Train Epoch: 45 [5120/60000 (9%)]	Loss: 0.038507
Train Epoch: 45 [6400/60000 (11%)]	Loss: 0.006922
Train Epoch: 45 [7680/60000 (13%)]	Loss: 0.026026
Train Epoch: 45 [8960/60000 (15%)]	Loss: 0.021902
Train Epoch: 45 [10240/60000 (17%)]	Loss: 0.001434
Train Epoch: 45 [11520/60000 (19%)]	Loss: 0.030899
Train Epoch: 45 [12800/60000 (21%)]	Loss: 0.001627
Train Epoch: 45 [14080/60000 (23%)]

Train Epoch: 48 [11520/60000 (19%)]	Loss: 0.010583
Train Epoch: 48 [12800/60000 (21%)]	Loss: 0.002839
Train Epoch: 48 [14080/60000 (23%)]	Loss: 0.001112
Train Epoch: 48 [15360/60000 (26%)]	Loss: 0.003576
Train Epoch: 48 [16640/60000 (28%)]	Loss: 0.056355
Train Epoch: 48 [17920/60000 (30%)]	Loss: 0.019286
Train Epoch: 48 [19200/60000 (32%)]	Loss: 0.001919
Train Epoch: 48 [20480/60000 (34%)]	Loss: 0.063519
Train Epoch: 48 [21760/60000 (36%)]	Loss: 0.017753
Train Epoch: 48 [23040/60000 (38%)]	Loss: 0.006403
Train Epoch: 48 [24320/60000 (41%)]	Loss: 0.007540
Train Epoch: 48 [25600/60000 (43%)]	Loss: 0.001789
Train Epoch: 48 [26880/60000 (45%)]	Loss: 0.004152
Train Epoch: 48 [28160/60000 (47%)]	Loss: 0.008774
Train Epoch: 48 [29440/60000 (49%)]	Loss: 0.007029
Train Epoch: 48 [30720/60000 (51%)]	Loss: 0.004810
Train Epoch: 48 [32000/60000 (53%)]	Loss: 0.035714
Train Epoch: 48 [33280/60000 (55%)]	Loss: 0.002607
Train Epoch: 48 [34560/60000 (58%)]	Loss: 0.002544
Train Epoch: 48 [35840/60000 (6

In [15]:
!rm -r mnist_data

'rm' is not recognized as an internal or external command,
operable program or batch file.
